<a href="https://colab.research.google.com/github/YorkJong/vistock/blob/feature%2Fibd/notebooks/ibd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install vistock from GitHub (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

In [3]:
%pip install "git+https://github.com/YorkJong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/YorkJong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-g72chvwz
  Running command git clone --filter=blob:none --quiet https://github.com/YorkJong/vistock.git /tmp/pip-req-build-g72chvwz
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/YorkJong/vistock.git to commit e8ed243f81cc294a1b2fa8c04da927fb6c7b9762
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.7 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.4.0-py3-none-any.whl size=41693 sha256=953b369a72fd4d267aafcf1a07be15304ae9f622de67fd9f302a3fdcc117fc2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-u6l25tve/wheels/3e/1c/74/97ec620813c1bb106679625869f7409ca5be8735d0a74457bf
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

### Setup and Configuration

In [4]:
# install cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [5]:
import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers


def rank(code, min_percentile=90, period='2y',
         tickers_getter=get_tickers, ref_ticker='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                           ref_ticker=ref_ticker)

    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    print('\nStock Rankings:')
    #display(rank_stock)
    top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
    display(top_stocks)
    num_rows, _ = top_stocks.shape
    print(f'\nnumber of filtered tickers: {num_rows}')
    top_stock_list = list(top_stocks["Ticker"])
    if tickers_getter is tw.get_tickers:
        top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
    print(top_stock_list)

    print('\n\nIndustry Rankings:')
    top_industries = rank_indust[rank_indust[ibd.TITLE_PERCENTILE] >= min_percentile]
    display(top_industries)
    num_rows, _ = top_industries.shape
    print(f'\nnumber of filtered sectors: {num_rows}')

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Save to CSV
    print("\n\n***")
    today = datetime.now().strftime('%Y%m%d')
    for table, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        table.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")


### Usage Explanation

##### Parameters
source (選擇要分析的股票來源):
- The source of stocks to analyze

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

period (歷史資料時間範圍)：
- The period for which to fetch historical data.

### IBD RS Rating and Ranking

In [ ]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'all'
}

rank(code_from_name[source], min_percentile, period)

ERROR:yfinance:BRK.B: No data found, symbol may be delisted
ERROR:yfinance:ARM: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:VLTO: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:$BF.B: possibly delisted; No price data found  (period=2y)


$BF.B: possibly delisted; No price data found  (period=2y)


ERROR:yfinance:SW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']
ERROR:yfinance:GEV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:SOLV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']



Stock Rankings:


,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
441,NVDA,Technology,Semiconductors,153.11,189.40,171.60,176.14,99,99,99,99,1
258,VST,Utilities,Utilities - Independent Power Producers,152.26,192.90,238.77,138.33,99,99,99,97,2
134,IRM,Real Estate,REIT - Specialty,147.73,123.27,113.08,105.08,99,95,85,71,3
429,FICO,Technology,Software - Application,143.04,130.98,118.38,131.71,99,97,91,96,4
204,GDDY,Technology,Software - Infrastructure,141.21,125.73,134.82,119.81,99,96,98,93,5
173,HWM,Industrials,Aerospace & Defense,135.80,122.79,137.10,114.55,99,94,98,88,6
3,MMM,Industrials,Conglomerates,135.23,104.05,111.20,83.24,98,79,82,16,7
181,TRGP,Energy,Oil & Gas Midstream,133.83,124.80,121.58,101.89,98,96,93,64,8
458,AXON,Industrials,Aerospace & Defense,132.28,99.95,114.31,113.13,98,72,88,86,9
92,GE,Industrials,Aerospace & Defense,130.72,124.68,146.89,121.87,98,95,98,94,10



number of filtered tickers: 52
['NVDA', 'VST', 'IRM', 'FICO', 'GDDY', 'HWM', 'MMM', 'TRGP', 'AXON', 'GE', 'UHS', 'KKR', 'TSM', 'NRG', 'ANET', 'MHK', 'NTAP', 'ISRG', 'AVGO', 'CEG', 'TYL', 'MPWR', 'EFX', 'K', 'CBRE', 'WELL', 'VTR', 'RCL', 'BRO', 'MSI', 'CTAS', 'MCK', 'EXR', 'IP', 'META', 'CFG', 'GS', 'TMUS', 'GEN', 'PGR', 'COST', 'EBAY', 'HCA', 'RTX', 'HIG', 'TT', 'FITB', 'PM', 'DHI', 'AMT', 'ICE', 'FSLR']


Industry Rankings:


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
84,Utilities - Independent Power Producers,Utilities,140.50,164.12,207.76,127.36,"VST,NRG",99,99,99,98,1
26,REIT - Healthcare Facilities,Real Estate,118.69,104.13,100.62,92.65,"WELL,VTR",98,92,64,33,2
12,Medical Care Facilities,Healthcare,118.25,100.85,109.08,106.12,"UHS,HCA,DVA",97,90,92,82,3
60,Oil & Gas Midstream,Energy,118.14,107.73,110.19,95.14,"TRGP,WMB,KMI,OKE",96,95,94,47,4
3,Conglomerates,Industrials,115.86,98.83,102.34,86.30,"MMM,HON",95,82,70,15,5
1,Tobacco,Consumer Defensive,114.31,98.55,97.81,84.08,"PM,MO",94,78,55,11,6
10,Residential Construction,Consumer Cyclical,111.94,100.00,107.38,112.54,"DHI,PHM,NVR,LEN",93,87,83,95,7
49,Consulting Services,Industrials,110.02,97.88,95.88,104.74,"EFX,VRSK",92,74,44,75,8
30,Computer Hardware,Technology,109.90,134.40,140.70,173.51,"ANET,NTAP,STX,HPQ,SMCI,WDC",91,98,98,99,9
28,Aerospace & Defense,Industrials,108.48,97.81,107.56,101.85,"HWM,AXON,GE,RTX,LMT,TDG,LHX,GD,NOC,HII,TXT,BA",90,73,87,65,10



number of filtered sectors: 10


***
Your "all_stocks_2y_20240807.csv" is in the "out" folder.
Your "all_industries_2y_20240807.csv" is in the "out" folder.
***



### IBD RS Rating and Ranking for Taiwan Stocks

In [6]:
from vistock import tw

source = "上市" #@param ["上市", "上櫃", "興櫃", "全部"]
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
period = "1y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '興櫃': 'ESB',
    '全部': 'all'
}

rank(code_from_name[source], min_percentile, period,
     tickers_getter=tw.get_tickers, ref_ticker='^TWII')

ERROR:yfinance:00939.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00940.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00941.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00943.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00944.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00945B.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00946.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00947.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00949.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:00951.TW: P


Stock Rankings:


,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1132,8374.TW,Industrials,Industrial Distribution,360.25,267.50,93.21,96.66,99.0,99,49,42,1
1012,6442.TW,Technology,Electronic Components,356.82,315.55,141.33,106.66,99.0,99,98,88,2
470,2365.TW,Technology,Computer Hardware,356.78,238.12,109.85,124.25,99.0,99,89,97,3
860,4562.TW,Industrials,Specialty Industrial Machinery,335.80,293.91,138.19,95.98,99.0,99,98,33,4
588,2543.TW,Industrials,Engineering & Construction,305.64,185.13,155.20,148.54,99.0,99,99,98,5
...,...,...,...,...,...,...,...,...,...,...,...,...
489,2393.TW,Technology,Electronic Components,118.82,108.61,114.25,99.22,90.0,87,93,66,115
1164,912000.TW,Technology,Communication Equipment,118.78,113.01,95.78,92.65,90.0,90,60,11,116
830,3712.TW,Technology,Semiconductors,118.22,115.43,95.90,106.20,90.0,90,61,87,117
805,3593.TW,Technology,Electronic Components,117.76,129.40,93.98,99.23,90.0,95,52,66,118



number of filtered tickers: 119
['羅昇', '光聖', '昆盈', '穎漢', '皇昌', '晶彩科', '海悅', '欣巴巴', '所羅門', '京城', '慧友', '弘憶股', '翔耀', '福懋油', '訊舟', '志聖', '喬福', '三地開發', '聯上發', '聯鈞', '華友聯', '藥華藥', '和椿', '錸德', '昇陽半導體', '迎廣', '波力-KY', '華城', '宏旭-KY', '益登', '大飲', '雲豹能源', '達能', '盟立', '鈺邦', '佳能', '一詮', '惠特', '愛山林', '直得', '晟銘電', '泰金寶-DR', '三發地產', '富世達', '新日興', '峰源-KY', '晟鈦', '德律', '潤弘', '佳必琪', '萬海', '振樺電', '時碩工業', '百達-KY', '瀚荃', '泓德能源-創', '瑞軒', '帝寶', '中砂', '東科-KY', '鉅祥', '亞翔', '中櫃', '台灣精銳', '百一', '銘鈺', '燿華', '達麗', '台火', '堤維西', '興泰', '隆銘綠能', '隆大', '亞力', '健策', '貿聯-KY', '聯策', '新美齊', '國產', '亞光', '元禎', '吉茂', '金寶', '精聯', '名軒', '櫻花建', '正崴', '千興', '藍天', '華固', '志信', '銘旺科', '喬山', '億泰', '天二科技', '鈞興-KY', '大量', '台積電', '三商', '國化', '華建', '嘉澤', '十銓', '寶徠', '辛耘', '同方友友-DR', '台灣虎航-創', '新建', '鴻海', '佳龍', '大亞', '圓剛', '上曜', '麗臺', '億光', '晨訊科-DR', '永崴投控', '力銘', '全坤建']


Industry Rankings:


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
45,Industrial Distribution,Industrials,180.37,150.91,94.91,96.87,"8374.TW,911608.TW,2373.TW",99.0,99,56,36,1
18,Real Estate - Development,Real Estate,151.68,129.50,115.44,103.80,"1436.TW,1805.TW",98.0,97,98,97,2
74,Utilities—Renewable,Utilities,132.78,137.83,103.94,108.22,"6869.TW,6873.TW,6806.TW",96.0,98,87,99,3
47,Engineering & Construction,Industrials,125.31,105.38,105.71,104.98,"2543.TW,9906.TW,2597.TW,6139.TW,3018.TW,5519.T...",95.0,88,89,98,4
22,Specialty Industrial Machinery,Industrials,119.30,110.33,101.44,101.80,"4562.TW,2467.TW,1540.TW,6215.TW,2464.TW,6706.T...",93.0,93,80,93,5
52,Real Estate—Development,Real Estate,118.63,106.01,109.54,99.09,"2524.TW,2537.TW,6177.TW,2442.TW,2539.TW,2548.T...",92.0,89,95,68,6
53,Real Estate—Diversified,Real Estate,111.91,113.39,114.25,101.49,"9946.TW,2520.TW,2545.TW,2547.TW",90.0,94,97,89,7



number of filtered sectors: 7


***
Your "TWSE_stocks_1y_20240808.csv" is in the "out" folder.
Your "TWSE_industries_1y_20240808.csv" is in the "out" folder.
***

